# HuggingFace Tutorial

This is a tutorial for me to learn how to use transformer with huggingface.

# Reference: 
- https://huggingface.co/
- https://huggingface.co/transformers/
- https://github.com/huggingface/datasets
- https://colab.research.google.com/drive/1IPkZo1Wd-DghIOK6gJpcb0Dv4_Gv2kXB?usp=sharing#scrollTo=ImupuGXDGq7b
- https://github.com/monologg/KoELECTRA
- https://github.com/monologg/KoELECTRA/blob/master/finetune/run_squad.py
- Korean Sentence Splitter: https://github.com/hyunwoongko/kss

# Installation

```bash
$ pip install transformers
```

In [1]:
import numpy as np

import torch
import torchtext

print(f"PyTorch Version: {torch.__version__}")
print(f"TorchText Version: {torchtext.__version__}")  

PyTorch Version: 1.6.0
TorchText Version: 0.8.0a0+c851c3e


# Datasets

need to install sentencepiece

```bash
$ pip install sentencepiece
$ pip install datasets
```

# How to use?

## Pipeline

- ConversationalPipeline
- FeatureExtractionPipeline
- FillMaskPipeline
- QuestionAnsweringPipeline
- SummarizationPipeline
- TextClassificationPipeline
- TextGenerationPipeline
- TokenClassificationPipeline
- TranslationPipeline
- ZeroShotClassificationPipeline
- Text2TextGenerationPipeline
- TableQuestionAnsweringPipeline

function: `pipeline`

- "feature-extraction": will return a FeatureExtractionPipeline.
- "sentiment-analysis": will return a TextClassificationPipeline.
- "ner": will return a TokenClassificationPipeline.
- "question-answering": will return a QuestionAnsweringPipeline.
- "fill-mask": will return a FillMaskPipeline.
- "summarization": will return a SummarizationPipeline.
- "translation_xx_to_yy": will return a TranslationPipeline.
- "text2text-generation": will return a Text2TextGenerationPipeline.
- "text-generation": will return a TextGenerationPipeline.
- "zero-shot-classification:: will return a ZeroShotClassificationPipeline.
- "conversation": will return a ConversationalPipeline.

model will be automatically downloaded in `~/.cache/huggingface/`

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", framework="pt")

In [ ]:
sentences = [
    "We are very happy to show you the 🤗 Transformers library.",
    "I'll go to Apple Store.",
    "This model covers a lot area. But, I won't use it. Since it is too hard to use."
]

In [ ]:
results = classifier(sentences)
for res in results:
    print(f"label: {res['label']}, with score: {round(res['score'], 4)}")

# Fine-Tuning with Custom Dataset

https://huggingface.co/transformers/custom_datasets.html#qa-squad


## Dataset


- 제목(title)
- 본문의 카테고리(source)
- 본문(context)
- 질문 번호(id)
- 육하원칙(classtype)
- 질문(question)
- 정답의 시작위치(answer_start)
- 정답(text)

In [2]:
import json
from tqdm import tqdm
from pathlib import Path
repo_path = Path().absolute().parent
data_path = repo_path.parent / "data" / "AIhub" / "QA"
for p in data_path.glob("ko*.json"):
    print(p)

/home/simonjisu/code/data/AIhub/QA/ko_nia_normal_squad_all.json
/home/simonjisu/code/data/AIhub/QA/ko_nia_clue0529_squad_all.json
/home/simonjisu/code/data/AIhub/QA/ko_nia_noanswer_squad_all.json


## Explore Datset

In [3]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(squad_dict["data"], total=len(squad_dict["data"]), desc="Reading Dataset"):
        for paragraph in group['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [4]:
train_file = "ko_nia_normal_squad_all.json"
train_path = data_path / train_file
val_file = "ko_nia_clue0529_squad_all.json"
val_path = data_path / val_file

train_contexts, train_questions, train_answers = read_squad(train_path)
val_contexts, val_questions, val_answers = read_squad(val_path)

Reading Dataset: 100%|██████████| 34500/34500 [00:00<00:00, 548757.41it/s]


In [5]:
print(len(train_contexts), len(val_contexts))

243425 96663


Let's see some samples

In [6]:
import termcolor

for idx in np.random.randint(0, len(train_contexts), size=(2,)):
    txt = train_answers[idx]["text"]
    context = train_contexts[idx].split(txt)
    context.insert(1, termcolor.colored(txt, "red", attrs=["bold"]))
    answer_end = train_answers[idx]['answer_start'] + len(train_answers[idx]['text'])  # not included like python range
    print(termcolor.colored("Context: ", attrs=["bold"]))
    print("".join(context))
    print(termcolor.colored("Question: ", attrs=["bold"]))
    print(train_questions[idx])
    print(termcolor.colored("Answer: ", attrs=["bold"]))
    print(f"  Start: {train_answers[idx]['answer_start']}, End: {answer_end}")
    print()

Context: 
댓글 조작 의혹 사건으로 구속된 김모(48·닉네임 드루킹)씨는 그동안 페이스북을 통해 추미애 더불어민주당 대표, 포털사이트 네이버, 문재인 대통령 핵심 지지자들인 ‘문꿀오소리’ 등을 싸잡아 비판했다. 이밖에도 여론 전문가, 남북 관계 전문가처럼 행세하면서 정치권에 훈수를 두는 일도 마다하지 않았다. 김씨는 자신의 페이스북 계정 ‘Sj Kim(드루킹)’을 통해 지난 1월 26일 “그동안 그렇게 하라고 해도 안하더니 네이버에서 드디어 계정 접속관리하고 기사 웹페이지를 손봤다”며 “기존 매크로 같은 것은 이틀 전부터 막혀서 안 될 것”이라고 언급했다. 매크로를 이용한 댓글 추천 조작 방법을 김씨가 명백하게 알고 있었다는 점을 뒷받침하는 글이라는 분석이 많다. 김씨는 이 글에서 추 대표도 비판했다. 그는 “청와대가 압력을 넣어 네이버 웹페이지를 개편하게 하면 뭘 하느냐”며 “‘문재앙’ 단어를 프레임화한 것은 그걸 기사화시킨 추 대표의 작품”이라고 지적했다. 이어 “지지자들은 열심히 댓글 방어하고 있는데 추 대표는 휴가 가셨다죠? 민주당의 앞날이 암울하다”고 썼다. ‘문꿀오소리’에 대한 혹평도 이어졌다. 김씨는 지난해 12월 페이스북 글에서 “자유한국당 댓글부대는 문 대통령 관련 기사에 악플을 단 뒤 순식간에 7000∼8000개의 추천을 찍는 화력”이라며 “문꿀오소리나 달빛기사단(문 대통령 핵심 지지층)은 기껏해야 그 반의 반에도 미치지 못한다”고 했다. 그러면서 “지금까지 문재인 지지자들은 온라인을 완전히 장악하고 있다고 오만에 빠져 있었다”고 썼다. 여론과 남북 관계에 대한 ‘점잖은’ 훈수도 빼놓지 않았다. 김씨는 1월 초 페이스북에서 “온라인 여론 점유율이 대통령 지지율이라고 여러 차례 이야기를 해도 정치인은 알아듣지 못한다”며 “아직도 오프라인 세상이 여론을 좌우한다고 생각하고 있다”고 했다. 이어 “통일은 반드시 이뤄야만 할 숙원”이라며 “그럴 때는 북한에 대한 발언도 예의있게 해야 한다. 요즘 20, 30대는 적대감 일색이라 걱정”이라고 했

In [7]:
def add_end_idx(answers, contexts):
    for idx, (answer, context) in enumerate(zip(answers, contexts)):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
            print(f"type1: {idx}")
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
            print(f"type2: {idx}")

In [8]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [9]:
from transformers import ElectraModel, ElectraTokenizer, ElectraTokenizerFast, ElectraForQuestionAnswering

tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")  
# Fast 를 써야 ._encodings 속성이 생긴다. 
# 안에는 Encoding class로 된 데이터가 list롤 있음
# train_encodings._encodings[0]
# Encoding(num_tokens=365, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
sample_len = 5
train_encodings = tokenizer(
    train_contexts[:sample_len], train_questions[:sample_len], padding = "max_length",
    max_length=512, truncation=True
)
val_encodings = tokenizer(
    val_contexts[:sample_len], val_questions[:sample_len], padding = "max_length",
    max_length=512, truncation=True
)

In [11]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))  # char_to_token: 문자가 몇 번째 토큰에 있는지 확인
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers[:sample_len])
add_token_positions(val_encodings, val_answers[:sample_len])

In [12]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [13]:
class ARGS:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args_dict = {
  "task": "korquad",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "KorQuAD_v1.0_train.json",
  "predict_file": "KorQuAD_v1.0_dev.json",
  "threads": 4,
  "version_2_with_negative": False,
  "null_score_diff_threshold": 0.0,
  "max_seq_length": 512,
  "doc_stride": 128,
  "max_query_length": 64,
  "max_answer_length": 30,
  "n_best_size": 20,
  "verbose_logging": True,
  "overwrite_output_dir": True,
  "evaluate_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "num_train_epochs": 7,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_type": "koelectra-base-v3",
  "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "output_dir": "koelectra-base-v3-korquad-ckpt",
  "seed": 42,
  "train_batch_size": 8,
  "eval_batch_size": 32,
  "logging_steps": 1000,
  "save_steps": 1000,
  "learning_rate": 5e-5
}
     
args = ARGS(**args_dict)

In [14]:
from transformers import ElectraForQuestionAnswering, ElectraConfig
config = ElectraConfig.from_pretrained(args.model_name_or_path)
model = ElectraForQuestionAnswering.from_pretrained(args.model_name_or_path, config=config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = "cpu"

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=args.train_batch_size, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()
    break

In [16]:
print("Inputs", input_ids.size())
print("Start Tokens", start_positions.size())
print("End Tokens", end_positions.size())
print("Loss", outputs.loss)
print("Start Logits", outputs.start_logits.size())
print("End Logits", outputs.end_logits.size())

Inputs torch.Size([5, 512])
Start Tokens torch.Size([5])
End Tokens torch.Size([5])
Loss tensor(6.0529, grad_fn=<DivBackward0>)
Start Logits torch.Size([5, 512])
End Logits torch.Size([5, 512])


## Is there more Efficient way???...

In [14]:
from transformers import squad_convert_examples_to_features
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [15]:
processor = SquadV2Processor()
examples = processor.get_train_examples(data_dir=data_path, filename="test.json")  # examples은 먼저 whitespace 기반으로 토크나이징함


100%|██████████| 5/5 [00:00<00:00, 643.59it/s]


In [16]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [17]:
features, train_dataset = squad_convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=512,
    doc_stride=128,
    max_query_length=64,
    is_training=True,
    return_dataset="pt",
    threads=4,
)


convert squad examples to features: 100%|██████████| 25/25 [00:00<00:00, 84.76it/s][A

add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 369216.90it/s]


In [18]:
a = features[1]
print(a.start_position, a.end_position)

26 30


In [19]:
for i, t in enumerate(a.tokens[:35]):
    print(i, t)

0 [CLS]
1 '
2 국제
3 청소년
4 ##포
5 ##럼
6 '
7 이
8 열리
9 ##는
10 때
11 ##는
12 ?
13 [SEP]
14 한국
15 ##청
16 ##소년단
17 ##체
18 ##협
19 ##의
20 ##회
21 ##와
22 여성
23 ##가족
24 ##부
25 ##는
26 22
27 ##일
28 ##부터
29 28
30 ##일
31 ##까
32 ##지
33 서울
34 ##과


In [20]:
class ARGS:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args_dict = {
  "task": "korquad",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "KorQuAD_v1.0_train.json",
  "predict_file": "KorQuAD_v1.0_dev.json",
  "threads": 4,
  "version_2_with_negative": False,
  "null_score_diff_threshold": 0.0,
  "max_seq_length": 512,
  "doc_stride": 128,
  "max_query_length": 64,
  "max_answer_length": 30,
  "n_best_size": 20,
  "verbose_logging": True,
  "overwrite_output_dir": True,
  "evaluate_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "num_train_epochs": 7,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_type": "koelectra-base-v3",
  "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "output_dir": "koelectra-base-v3-korquad-ckpt",
  "seed": 42,
  "train_batch_size": 8,
  "eval_batch_size": 32,
  "logging_steps": 1000,
  "save_steps": 1000,
  "learning_rate": 5e-5
}
     
args = ARGS(**args_dict)

def tolist(tensor):
    return tensor.detach().cpu().tolist()

In [21]:
from transformers import ElectraForQuestionAnswering, ElectraConfig
config = ElectraConfig.from_pretrained(args.model_name_or_path)
model = ElectraForQuestionAnswering.from_pretrained(args.model_name_or_path, config=config)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.weight'

In [24]:
device = "cpu"

In [22]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.train_batch_size)
t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total
)

global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()

In [25]:
model.to(device)
model.train()
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)

    inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
        "start_positions": batch[3],
        "end_positions": batch[4],
    }
    break

In [26]:
input_ids=inputs["input_ids"]
attention_mask=inputs["attention_mask"]
token_type_ids=inputs["token_type_ids"]

In [27]:
o = model.electra.forward(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
o.last_hidden_state.size()

torch.Size([8, 512, 768])

In [28]:
fin_o = model.qa_outputs(o.last_hidden_state)
fin_o.size()

torch.Size([8, 512, 2])

In [29]:
outputs = model(**inputs)
print(f"start predict {outputs.start_logits.argmax(1)}")
print(f"start answer {inputs['start_positions']}")
print(f"start predict {outputs.end_logits.argmax(1)}")
print(f"start answer {inputs['end_positions']}")

start predict tensor([374, 413, 331, 298,  64, 502, 295, 411])
start answer tensor([  0,  43,   0, 103,  64,  87, 114, 308])
start predict tensor([253, 189, 132, 260,   1, 374, 381,  45])
start answer tensor([  0,  43,   0, 112,  84,  91, 116, 317])


**eval phase**

In [30]:
eval_examples = processor.get_dev_examples(data_dir=data_path, filename="test.json")  # examples은 먼저 whitespace 기반으로 토크나이징함
eval_features, eval_dataset = squad_convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=512,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    return_dataset="pt",
    threads=4,
)


100%|██████████| 5/5 [00:00<00:00, 659.09it/s]

convert squad examples to features: 100%|██████████| 25/25 [00:00<00:00, 86.09it/s][A

add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 308404.71it/s]


In [31]:
# for fea in eval_features:
#     fea.unique_id -= 1000000000
eval_dataloader = DataLoader(eval_dataset, shuffle=False, batch_size=args.train_batch_size)

In [32]:
all_results = []
for batch in eval_dataloader:
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }
        example_indices = batch[3]
        outputs = model(**inputs)
        
    for i, example_index in enumerate(example_indices):
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        output = [tolist(o[i]) for o in outputs.values()]
        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)
        all_results.append(result)

In [33]:
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate
)

In [34]:
output_prediction_file = "./predictions.json"
output_nbest_file = "./nbest_predictions.json"
output_null_log_odds_file = "./null_odds.json"

In [35]:
predictions = compute_predictions_logits(
    eval_examples,
    eval_features,
    all_results,
    args.n_best_size,
    args.max_answer_length,
    args.do_lower_case,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    args.verbose_logging,
    args.version_2_with_negative,
    args.null_score_diff_threshold,
    tokenizer,
)

In [36]:
for i, j in zip(eval_examples, predictions.items()):
    if i.qas_id == j[0]:
        print(i.qas_id)
        print(f"Answer: {i.answers[0]['text']}")
        print(f"Predict: {j[1]}")
        print()
    else:
        print(i.qas_id, j[0])

c1_57059-1
Answer: 한국청소년단체협의회와 여성가족부
Predict: 선언문을 전달할 예정이다

c1_57060-1
Answer: 22일부터 28일
Predict: 선언문을 전달할 예정이다

c1_57061-1
Answer: '청소년과 뉴미디어'
Predict: 선언문을 전달할 예정이다

c1_57062-1
Answer: 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다.
Predict: 선언문을 전달할 예정이다

m5_306705-1
Answer: 샐리
Predict: 육상 볼링 양궁 리듬체조 에어로빅 선수권대회(이하 아육대)'에서는

c1_151305-1
Answer: 보조 교통 경찰로 일하는 천중핑
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151306-1
Answer: 지난달 28일
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151307-1
Answer: 구이저우성 카일리시
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151308-1
Answer: ‘중국의 좋은 이웃상’과 함께 상금 1만 위안(약 170만원)을 수여
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151309-1
Answer: 이틀 간의 코마 상태 이후 의식을 회복해 지난 2일부터 중환자실에서 치료를 받고 있습니다
Predict: 아이는 죽었을 것”이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다

c1_151310-1
Answer: 열쇠공이 문을 따는 소리에 겁을 먹고 창문 밖으로 도망을 치려다
Predict: 회복해 지난

c1_151311-1
Answer: 아이가 잠든 사이 돌보던 아이의 할머니가 쓰레기를 버리러 나갔다가 문이 잠기는 바람에 열쇠공을 불렀던 것
Predict: 아이는 죽었을 것”이

In [37]:
results = squad_evaluate(eval_examples, predictions)

In [38]:
results

OrderedDict([('exact', 0.0),
             ('f1', 5.556363636363637),
             ('total', 25),
             ('HasAns_exact', 0.0),
             ('HasAns_f1', 5.556363636363637),
             ('HasAns_total', 25),
             ('best_exact', 0.0),
             ('best_exact_thresh', 0.0),
             ('best_f1', 5.556363636363637),
             ('best_f1_thresh', 0.0)])

Why there is 0 to predict?

In [69]:
vocab_rev = {v: k for k, v in tokenizer.vocab.items()}
tostring = lambda x: " ".join(x).replace(" ##", "").replace("[PAD]", "").strip()
def show_original(inputs):
    tokens = [vocab_rev[i.item()] for i in inputs["input_ids"]]
    s, e = inputs["start_positions"].item(), inputs["end_positions"].item()
    print(s, e)
    print("Answer: ", tostring(tokens[s:(e+1)]))
    print(tostring(tokens))

In [78]:
idxes = [5, 6, 7, 8, 9, 10]
for i, idx in enumerate(idxes):
    batch = train_dataset[idx]
    inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "token_type_ids": batch[2],
        "start_positions": batch[3],
        "end_positions": batch[4],
    }
    if i % 2 == 0:
        print("----------------------------"*3)
    show_original(inputs)
    print()

------------------------------------------------------------------------------------
103 112
Answer:  보조 교통 경찰로 일하는 천중핑
[CLS] 중국에서 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠진 사람은 누구야 ? [SEP] 중국의 한 여성 경찰이 아파트에서 추락하던 3세 아이를 살리고 자신은 혼수상태에 빠졌습니다 . 의인 ( 義 人 ) 의 소식이 알려지자 각박한 중국 사회에 큰 반향을 일으키고 있습니다 . 5일 귀주도시망 등 중국 현지 언론에 따르면 구이저우성 카일리시에 보조 교통 경찰로 일하는 천중핑 ( 49 ) 은 지난달 28일 한 아파트에서 비상 상황이 발생했다는 연락을 받고 현장으로 향했습니다 . 도착했을 때 아파트 4층 창문에서 여자 아이가 매달려 있었습니다 . 곧이어 아이는 손에 힘이 빠지면서 밑으로 추락했습니다 . 천중핑과 다른 세명의 이웃들이 달려갔습니다 . 그리고 아이는 바닥이 아니라 천중핑의 팔에 떨어졌습니다 . 중간 비막이 천막 때문에 속도가 줄기는 했지만 추락의 충격은 천중핑이 고스란히 감당해야 했습니다 . 아이는 즉시 병원으로 옮겨져 치료를 받았습니다 . 다리 골절로 그리 심각한 상황은 아니라고 합니다 . 하지만 생명의 은인이자 영웅은 커다란 댓가를 치러야 했다 . 뇌출혈로 인한 의식불명 상태에 빠진 것이다 . 다행히 이틀 간의 코마 상태 이후 의식을 회복해 지난 2일부터 중환자실에서 치료를 받고 있습니다 . 아이는 열쇠공이 문을 따는 소리에 겁을 먹고 창문 밖으로 도망을 치려다 사고를 당한 것으로 전해졌습니다 . 아이가 잠든 사이 돌보던 아이의 할머니가 쓰레기를 버리러 나갔다가 문이 잠기는 바람에 열쇠공을 불렀던 것입니다 . 아이의 엄마는 “ 천중핑의 도움이 없었다면 아이는 죽었을 것 ” 이라며 딸을 구해준 천중핑에게 감사의 뜻을 전했습다 . 카일리시 정부 대표와 공안부 관계자들도 천중핑이 입원한 병원을 찾아 위로하고 회복될때까지 도움

If over 512 tokens, it will generate sentences from the back like: `tokens[-512:]`

In [1]:
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torchmetrics

from transformers import (
    ElectraForQuestionAnswering, 
    ElectraConfig, 
    ElectraTokenizer,
    AdamW,
    squad_convert_examples_to_features,
    get_linear_schedule_with_warmup
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate
)

In [2]:
train_file = "ko_nia_normal_squad_all.json"
val_file = "ko_nia_clue0529_squad_all.json"

repo_path = Path().absolute().parent
data_path = repo_path.parent / "data" / "AIhub" / "QA"
ckpt_path = repo_path.parent / "ckpt"
if not ckpt_path.exists():
    ckpt_path.mkdir()
else:
    for x in ckpt_path.glob("*"):
        if x.is_dir():
            x.rmdir()
        else:
            x.unlink()
    ckpt_path.rmdir()
    ckpt_path.mkdir()

In [3]:
train_file = "test.json"
val_file = "test.json"
args_dict = {
    "task": "AIHub_QA",
    "data_path": data_path,
    "ckpt_path": ckpt_path,
    "train_file": train_file,
    "val_file": val_file,
    "cache_file": "{}_cache",
    "random_seed": 77,
    "threads": 4,
    "version_2_with_negative": False,
    "null_score_diff_threshold": 0.0,
    "max_seq_length": 512,
    "doc_stride": 128,
    "max_query_length": 64,
    "max_answer_length": 30,
    "n_best_size": 20,
    "verbose_logging": True,
    "do_lower_case": False,
    "num_train_epochs": 50,
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "warmup_proportion": 0,
    "model_type": "koelectra-base-v3",
    "model_name_or_path": "monologg/koelectra-base-v3-discriminator",
    "output_dir": "koelectra-base-v3-korquad-ckpt",
    "seed": 42,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "learning_rate": 5e-5,
    "output_prediction_file": "predictions_{}.json",
    "output_nbest_file": "nbest_predictions_{}.json",
    "output_null_log_odds_file": "null_odds_{}.json",
}

In [4]:
class Model(pl.LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters() 
        self.config = ElectraConfig.from_pretrained(self.hparams.model_name_or_path)
        self.model = ElectraForQuestionAnswering.from_pretrained(
            self.hparams.model_name_or_path, 
            config=self.config
        )
        self.tokenizer = ElectraTokenizer.from_pretrained(self.hparams.model_name_or_path)
        # create dataset and cache it
        self.create_dataset(state="train")
        self.create_dataset(state="val")
        self.eval_examples, self.eval_features = self.load_cache(state="val", return_dataset=False)
        # function
        self.tolist = lambda x: x.detach().cpu().tolist()
        
    def create_dataset(self, state:str="train"):
        r"""
        Args:
            state: train or val
        """
        processor = SquadV2Processor()
        if state == "train":
            examples = processor.get_train_examples(
                data_dir=self.hparams.data_path, 
                filename=self.hparams.train_file
            )
            is_training = True
        elif state == "val":
            examples = processor.get_dev_examples(
                data_dir=self.hparams.data_path, 
                filename=self.hparams.val_file
            )
            is_training = False
        else:
            raise ValueError("state should be train or val")
            
        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=self.tokenizer,
            max_seq_length=self.hparams.max_seq_length,
            doc_stride=self.hparams.doc_stride,
            max_query_length=self.hparams.max_query_length,
            is_training=is_training,
            return_dataset="pt",
            threads=self.hparams.threads,
        )
        # https://huggingface.co/transformers/_modules/transformers/data/processors/squad.html
        # TODO: Cannot find the key unique_id
        # BUG: must set argument of `trainer: num_sanity_val_steps=0` to avoid error.

        cache = dict(dataset=dataset, examples=examples, features=features)
        torch.save(cache, self.hparams.ckpt_path / self.hparams.cache_file.format(state))
        print(f"[INFO] cache file saved! {self.hparams.ckpt_path / self.hparams.cache_file.format(state)}")

    def load_cache(self, state:str="train", return_dataset:bool=True):
        cache = torch.load(self.hparams.ckpt_path / self.hparams.cache_file.format(state))
        dataset, examples, features = cache["dataset"], cache["examples"], cache["features"]

        if return_dataset:
            return dataset
        else:
            return examples, features

    def forward(self, **kwargs):
        return self.model(**kwargs)

    def training_step(self, batch, batch_idx):
        inputs_ids, attention_mask, token_type_ids, start_positions, end_positions, *_ = batch

        outputs = self(
            input_ids=inputs_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions=start_positions,
            end_positions=end_positions
        )

        loss = outputs.loss
        return  {'loss': loss}

    def validation_step(self, batch, batch_idx):
        inputs_ids, attention_mask, token_type_ids, example_indices, *_ = batch
        
        outputs = self(
            input_ids=inputs_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            start_positions=None,
            end_positions=None
        )
        
        batch_results = []
        
        for i, example_index in enumerate(example_indices):
            eval_feature = self.eval_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            output = [self.tolist(o[i]) for o in outputs.values()]
            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)
            batch_results.append(result)
            
        return batch_results
    
    def train_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for out in outputs:
            loss += out["loss"].detach().cpu()
        loss = loss / len(outputs)

        return {'loss': loss}

    def validation_epoch_end(self, outputs):
        all_results = []
        for res in outputs:
            all_results += res

        predictions = compute_predictions_logits(
            self.eval_examples,
            self.eval_features,
            all_results,
            self.hparams.n_best_size,
            self.hparams.max_answer_length,
            self.hparams.do_lower_case,
            self.hparams.ckpt_path / self.hparams.output_prediction_file.format(self.global_step),
            self.hparams.ckpt_path / self.hparams.output_nbest_file.format(self.global_step),
            self.hparams.ckpt_path / self.hparams.output_null_log_odds_file.format(self.global_step),
            self.hparams.verbose_logging,
            self.hparams.version_2_with_negative,
            self.hparams.null_score_diff_threshold,
            self.tokenizer,
        )
        results = squad_evaluate(self.eval_examples, predictions)
        accuracy = results["exact"]
        f1 = results["f1"]
        self.log("accuracy", accuracy, on_epoch=True, prog_bar=True)
        self.log("f1", f1, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        t_total = self.total_steps()
        
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 
                "weight_decay": 0.0
            },
        ]
        optimizer = AdamW(
            params=optimizer_grouped_parameters, 
            lr=self.hparams.learning_rate, 
            eps=self.hparams.adam_epsilon
        )
        scheduler = get_linear_schedule_with_warmup(
            optimizer=optimizer, 
            num_warmup_steps=int(t_total * self.hparams.warmup_proportion), 
            num_training_steps=t_total
        )
        
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def create_dataloader(self, state:str="train"):
        r"""
        Args:
            state: train or val
        """
        if state == "train":
            shuffle = True
            batch_size = self.hparams.train_batch_size
        elif state == "val":
            shuffle = False
            batch_size = self.hparams.eval_batch_size
        else:
            raise ValueError("state should be train or val")
        dataset = self.load_cache(state, return_dataset=True)
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=self.hparams.threads
        )
        return dataloader

    def train_dataloader(self):
        return self.create_dataloader(state="train")

    def val_dataloader(self):
        return self.create_dataloader(state="val")
    
    def total_steps(self):
        r"""
        source: https://github.com/PyTorchLightning/pytorch-lightning/issues/1038
        """
        return len(self.train_dataloader()) * self.hparams.num_train_epochs


In [5]:
def main(args_dict):
    print("[INFO] Using PyTorch Ver", torch.__version__)
    print("[INFO] Seed:", args_dict["random_seed"])
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        filename="epoch{epoch}-f1{f1:.4f}",
        monitor="f1",
        save_top_k=3,
        mode="max",
    )
    pl.seed_everything(args_dict["random_seed"])
    model = Model(**args_dict)
    
    print("[INFO] Start FineTuning")
    trainer = pl.Trainer(
        callbacks=[checkpoint_callback],
        max_epochs=args_dict["num_train_epochs"],
        deterministic=torch.cuda.is_available(),
        gpus=-1 if torch.cuda.is_available() else None,
    )
    trainer.fit(model)

In [ ]:
main(args_dict)

---